准备数据集：<br>
annotations/person_keypoints_val2017.json是5000张图片的标注信息<br>
images下是5000张图片<br>

In [2]:
"""解析coco数据集的标注信息
"""
import json
import os
json_path = "annotations/person_keypoints_val2017.json"
json_labels = json.load(open(json_path, "r"))
annotations = json_labels['annotations'] # list
images = json_labels['images'] # list
categories = json_labels['categories'] # list

In [2]:
# 存储图片名和图片id之间的映射关系
idtoimage = {}
for image in images:
    file_name = image['file_name']
    image_id = image['id'] 
    height = image['height'] 
    width = image['width'] 
    idtoimage[image_id] = [file_name,height,width]

In [ ]:
# 存储类别名和类别id之间的映射关系。person此处对应1，需要用CLASSES转换一下
idtoclss = {}
for category in categories:
    id = category['id']
    name = category['name'] # 类别名
    idtoclss[id] = name
CLASSES = ['person']

In [7]:
# 解析人体关键点：<class-index> <x> <y> <width> <height> <px1> <py1> <px2> <py2> ... <pxn> <pyn>,归一化
Dim = 3
for annotation in annotations:
    try:
        count = annotation['num_keypoints'] # 关键点标注数量
        if count > 0:
            image_id = annotation['image_id']  
            category_id = annotation['category_id']  # 类别编号
            bbox = annotation['bbox'] # 左上角x,y,w,h
            # segmentation = annotation['segmentation'][0]  # 分割点
            keypoints = annotation['keypoints']  # 人体关键点
                    
            classname = idtoclss[category_id] # 类别名
            category_id = CLASSES.index(classname) # 转下编号
            
            file = idtoimage[image_id]
            filename,h,w = file[0],file[1],file[2]
            
            # bbox归一化
            cx = (bbox[0] + bbox[2]/2) / w
            cy = (bbox[1] + bbox[3]/2) / h
            box_w, box_h = bbox[2]/w, bbox[3]/h
            line = [str(i) for i in [category_id, cx, cy, box_w, box_h]]
            line = ' '.join(line)
        
            # keypoints归一化
            x = [i/w for i in keypoints[0::3]] # x坐标归一化
            y = [i/h for i in keypoints[1::3]] # y坐标归一化
            v = [i for i in keypoints[2::3]] # v=0没标注，v=1被遮挡,v=2可见
            xy = ''
            if Dim == 2:
                for i in range(len(x)):
                    xy += str(x[i]) + ' ' + str(y[i]) + ' '
            if Dim == 3:
                for i in range(len(x)):
                    xy += str(x[i]) + ' ' + str(y[i]) + ' ' + str(v[i]) + ' '
            line = line + ' ' + xy + '\n'
            outfile = filename.split('.')[0]+'.txt'
            outfile = os.path.join('labels_person_keypoints',outfile)
            with open(outfile,'a') as f:
                f.write(line)
    except:
        continue

制作一个500张图片的小样本集

In [10]:
os.mkdir('labels_2d_500')
os.mkdir('labels_3d_500')
os.mkdir('images500')

In [11]:
import shutil
names = os.listdir('labels_person_keypoints_2d')[:500]
for name in names:
    cur_path = os.path.join('labels_person_keypoints_2d', name)
    to_path = os.path.join('labels_2d_500', name)
    shutil.copy(cur_path, to_path)  
    
    cur_path = os.path.join('labels_person_keypoints_3d', name)
    to_path = os.path.join('labels_3d_500', name)
    shutil.copy(cur_path, to_path)  
    
    image = name.split('.')[0]+'.jpg'
    cur_path = os.path.join('images', image)
    to_path = os.path.join('images500', image)
    shutil.copy(cur_path, to_path)

In [3]:
# 划分训练集和测试集
ftrain = open('train.txt', 'w')
fval = open('val.txt', 'w')
imgaes = os.listdir('images')
for i,image in enumerate(imgaes):
    name = f'/home/data/images/{image}\n'
    if i % 10 == 2:
        fval.write(name)
    else:
        ftrain.write(name)
ftrain.close()
fval.close()